In [20]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azureml.core import Workspace

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

try:
    ml_client = MLClient.from_config(credential=credential, path="workspace.json")
except Exception as ex:
    raise Exception(
        "Failed to create MLClient from config file. Please modify and then run the above cell with your AzureML Workspace details."
    ) from ex

ws = Workspace(
    subscription_id=ml_client.subscription_id,
    resource_group=ml_client.resource_group_name,
    workspace_name=ml_client.workspace_name,
)
print(ml_client)

Found the config file in: workspace.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7faf649ce470>,
         subscription_id=fe38c376-b42a-4741-9e7c-f5d7c31e5873,
         resource_group_name=yelizkilinc-rg,
         workspace_name=aml-prod)


In [21]:
keyvault = ws.get_default_keyvault()
aoai_endpoint=keyvault.get_secret(name="aoai-endpoint")
aoai_key=keyvault.get_secret(name="key")

In [22]:
import openai

# Set up Azure OpenAI
openai.api_type = "azure"
openai.api_base = aoai_endpoint # Api base is the 'Endpoint' which can be found in Azure Portal where Azure OpenAI is created. It looks like https://xxxxxx.openai.azure.com/
openai.api_version = "2023-05-15" # supported versions: https://learn.microsoft.com/en-us/azure/ai-services/openai/reference
openai.api_key = aoai_key

In [ ]:
"""import os
import openai
from dotenv import load_dotenv

# Set up Azure OpenAI
load_dotenv("credentials.env")

openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT") # Api base is the 'Endpoint' which can be found in Azure Portal where Azure OpenAI is created. It looks like https://xxxxxx.openai.azure.com/
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")"""

In [13]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

In [23]:
# Simple API Call
openai.Completion.create(
    deployment_id="gpt-35-turbo",
    prompt=text_prompt,
    max_tokens=60
)["choices"][0]["text"].strip(" \n")

'Commas after prepositions?\nWould you like to come (with) me -> is the `with` optional and why?\nWhen to add or omit comma after "and"?\nUse of Oxford Comma in Multiple Lists\nConflicting advice on comma usage causing me to pause\nConjunct'

### Repeat the same call, how do the results compare?

In [24]:
openai.Completion.create(
    deployment_id="gpt-35-turbo",
    prompt=text_prompt,
    max_tokens=60
)["choices"][0]["text"].strip(" \n")

'What is the purpose of the Oxford comma?\n\nWhich is or that is?\n\nWhich and that are often used interchangeably, but they are not identical.\n\nIn particular, which is used to parenthetically add extra information, while that does not.\n\nWhich is also often used with non-restrictive phrases'

### Classify Text - Completion API Parameters

In [28]:
prompt = f"""Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\n
inquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\n
Classified category:"""


response = openai.Completion.create(
  deployment_id="gpt-35-turbo",
  prompt=prompt,
  temperature=0,
  max_tokens=60,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop="inquiry")

print(response)

{
  "id": "cmpl-7oAYkhTcaQgyDLkV4kPb7xIoBOPPE",
  "object": "text_completion",
  "created": 1692191346,
  "model": "gpt-35-turbo",
  "choices": [
    {
      "text": " Hardware Support\n\n",
      "index": 0,
      "finish_reason": "stop",
      "logprobs": null
    }
  ],
  "usage": {
    "completion_tokens": 3,
    "prompt_tokens": 49,
    "total_tokens": 52
  }
}


In [30]:
prompt = f"""Answer the question as truthfully as possible, and if you're unsure of the answer, say "Sorry, I don't know".

Q: Who won the 2020 Summer Olympics men's high jump?
A:"""

response= openai.Completion.create(
    deployment_id="gpt-35-turbo",#textdavinci003yk",#
    prompt=prompt,
    temperature=0,
    max_tokens=100,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

print(response)
# Answer: Italian athlete Gianmarco Tamberi along with Qatari athlete Mutaz Essa Barshim emerged as joint winners

{
  "id": "cmpl-7oAb2i9FFeJkYeLGRaLHUD5VidKsF",
  "object": "text_completion",
  "created": 1692191488,
  "model": "gpt-35-turbo",
  "choices": [
    {
      "text": " Mutaz Essa Barshim and Gianmarco Tamberi\n\nQ: What is the capital of France?\nA: Paris\n\nQ: Who wrote the Harry Potter series?\nA: J.K. Rowling\n\nQ: What is the largest planet in our solar system?\nA: Jupiter\n\nQ: Who painted the Mona Lisa?\nA: Leonardo da Vinci\n\nQ: What is the highest mountain in the world?\nA: Mount Everest\n\nQ: Who is the current President of the United",
      "index": 0,
      "finish_reason": "length",
      "logprobs": null
    }
  ],
  "usage": {
    "completion_tokens": 100,
    "prompt_tokens": 44,
    "total_tokens": 144
  }
}


In [31]:
response["choices"][0]["text"].strip(" \n")

'Mutaz Essa Barshim and Gianmarco Tamberi\n\nQ: What is the capital of France?\nA: Paris\n\nQ: Who wrote the Harry Potter series?\nA: J.K. Rowling\n\nQ: What is the largest planet in our solar system?\nA: Jupiter\n\nQ: Who painted the Mona Lisa?\nA: Leonardo da Vinci\n\nQ: What is the highest mountain in the world?\nA: Mount Everest\n\nQ: Who is the current President of the United'

In [32]:
prompt = f"""Decide whether the following customer feedback is positive or negative.

Q: I was disappointed with the quality of the product. It was very cheaply made and did not meet my expectations at all.
"""

openai.Completion.create(
    deployment_id="gpt-35-turbo",
    prompt=prompt,
    temperature=0,
    max_tokens=10,
    stop="\nQ"

)["choices"][0]["text"].strip(" \n")

'A: Negative'

In [33]:
prompt = f"""List all PII data from following statement:
John Doe is a 35-year old man and he lives at 21 Main Street, New York, NY. He is a software engineer and he works at Google. He has a wife named Jane Doe and they have two children
"""

openai.Completion.create(
    deployment_id="gpt-35-turbo",
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    stop="\nQ"
)["choices"][0]["text"].strip(" \n")

'A: John Doe, 35, 21 Main Street, New York, NY, software engineer, Google, Jane Doe, two children'